In [13]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer

from matplotlib import pyplot as plt

In [77]:
train = pd.read_csv('salary-train.csv')

In [78]:
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [79]:
train['FullDescription'] = train['FullDescription'].str.lower()

In [80]:
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager// luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [81]:
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [82]:
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [83]:
vectorizer = TfidfVectorizer(min_df=5)

In [84]:
X_vect = vectorizer.fit_transform(train.FullDescription)

In [85]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

In [86]:
from sklearn.feature_extraction import DictVectorizer

In [87]:
enc = DictVectorizer()
train_categ = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [88]:
from scipy.sparse import hstack

In [89]:
X = hstack([X_vect, train_categ])

In [90]:
from sklearn.linear_model import Ridge

In [91]:
model = Ridge(alpha=1, random_state=241, fit_intercept=False, solver='lsqr')
model.fit(X, train.SalaryNormalized)

Ridge(alpha=1, copy_X=True, fit_intercept=False, max_iter=None,
   normalize=False, random_state=241, solver='lsqr', tol=0.001)

In [92]:
train_test = pd.read_csv('salary-test-mini.csv')

In [93]:
train_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [94]:
train_test['FullDescription'] = train_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [95]:
train_test['FullDescription'] = train_test['FullDescription'].str.lower()

In [96]:
train_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [97]:
X_vect_test = vectorizer.transform(train_test['FullDescription'])

In [99]:
train_test['LocationNormalized'].fillna('nan', inplace=True)
train_test['ContractTime'].fillna('nan', inplace=True)

In [100]:
train_categ = enc.transform(train_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [101]:
X_test = hstack([X_vect_test, train_categ])

In [102]:
X_test

<2x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [103]:
model.predict(X_test)

array([57130.38135181, 37435.4100617 ])